# 모두를 위한 딥러닝 : Pytorch
## Lecture 11-2 - RNN charseq
----
본 문서는 모두를 위한 딥러닝 강의를 듣고 공부 목적으로 작성되었습니다.

### RNN의 특징
------
* RNN은 활성화함수로 주로 sigmoid나 tahh 함수가 쓰인다. 이는 Relu를 쓰게 되면 계속해서 양수값이 여러번 곱해지며, 기울기 폭발이 발생할 가능성이 높기 때문이다.  
* one-hot encoding을 사용하는 주된 이유는 크기가 같기 때문이다. 각각의 문자에 인덱스로 의미를 부여하면, 문자 하나하나에 다른 크기가 배정되는 것이므로, 의미가 부여될 수 있기 때문이다.

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
torch.manual_seed(0)

In [3]:
sample = ' if you want you'

In [4]:
char_set = list(set(sample))
char_dic = {c : i for i,c in enumerate(char_set)}
print(char_dic)

{'n': 0, 'u': 1, 'y': 2, 't': 3, ' ': 4, 'w': 5, 'f': 6, 'a': 7, 'o': 8, 'i': 9}


In [5]:
dic_size = len(char_dic)
hidden_size = len(char_dic)
learning_rate = 0.1

In [13]:
sample_idx = [char_dic[c] for c in sample]
x_data = sample_idx[:-1]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
y_data = sample_idx[1:]

In [14]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print(X.shape)
print(Y.shape)

torch.Size([15, 10])
torch.Size([15])


In [8]:
rnn = torch.nn.RNN(dic_size, hidden_size, batch_first=True)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)

In [17]:
for i in range(50):
    optimizer.zero_grad()
    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1,dic_size),Y)
    loss.backward()
    optimizer.step()

    result = torch.max(outputs, dim=1)[1]
    result_str = ''.join([char_set[c] for c in result])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.9885978698730469 prediction:  tensor([5, 6, 4, 5, 8, 1, 4, 5, 1, 4, 5, 7, 7, 8, 1]) true Y:  [9, 6, 4, 2, 8, 1, 4, 5, 7, 0, 3, 4, 2, 8, 1] prediction str:  wf wou wu waaou
1 loss:  1.7345199584960938 prediction:  tensor([2, 6, 4, 5, 8, 1, 4, 5, 1, 4, 2, 2, 2, 8, 1]) true Y:  [9, 6, 4, 2, 8, 1, 4, 5, 7, 0, 3, 4, 2, 8, 1] prediction str:  yf wou wu yyyou
2 loss:  1.5653952360153198 prediction:  tensor([2, 6, 4, 2, 8, 1, 4, 5, 7, 4, 3, 4, 5, 8, 1]) true Y:  [9, 6, 4, 2, 8, 1, 4, 5, 7, 0, 3, 4, 2, 8, 1] prediction str:  yf you wa t wou
3 loss:  1.4270879030227661 prediction:  tensor([2, 6, 4, 2, 8, 1, 4, 5, 7, 4, 3, 4, 2, 8, 1]) true Y:  [9, 6, 4, 2, 8, 1, 4, 5, 7, 0, 3, 4, 2, 8, 1] prediction str:  yf you wa t you
4 loss:  1.3238537311553955 prediction:  tensor([2, 6, 4, 2, 8, 1, 4, 5, 7, 4, 3, 4, 2, 8, 1]) true Y:  [9, 6, 4, 2, 8, 1, 4, 5, 7, 0, 3, 4, 2, 8, 1] prediction str:  yf you wa t you
5 loss:  1.2394696474075317 prediction:  tensor([2, 6, 4, 2, 8, 1, 4, 5, 7, 4, 3, 4, 

8번째 부터 학습이 잘 진행된 것을 확인할 수 있다.